In [3]:
#***********************************#
#                                   #
#             Packages              #        
#                                   #
#***********************************#
import dsp_py as dsp

import csv
import allantools as allan
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import allantools as AT
import ipywidgets as wdg
from scipy import signal as sig  
#import panel as pn

#pn.extension()

In [4]:
#***********************************#
#                                   #
#             Constants             #        
#                                   #
#***********************************#

adc_clk = 512 * 10 ** 6 # adc clock rate

T_adc = 1/(adc_clk) # adc period

fpga_clk = 256 * 10 ** 6 # fpga fabric clock rate

FFT_length = 1024 # Length of FFT used in spectrometer

accum_len = 2 ** 23 # number of clock cycles of accumulation in every dump of data

accum_time = ((adc_clk)/FFT_length)/accum_len # amount of time one accumulation cycle integrates (seconds)



# Put parameters for testing here

source_freq = 100 * 10 ** 6 
square_freq = 100 * 10 ** 3

In [5]:
#***********************************#
#                                   #
#            Functions              #        
#                                   #
#***********************************#


################################################
# Wave generation function:                    #
# Input: frequency and amount of time          #
# Output : I and Q arrays for wave             #
################################################

def cool_wave(amp, freq, time):
    omega = 2 * np.pi * freq    
    wave = amp * np.exp(1j * omega * time)
    i = np.real(wave)
    q = np.imag(wave)
    return i,q
    
def real_wave(amp, freq, time, phase=0):
    omega = 2 * np.pi * freq    
    wave = amp*np.cos(omega * time + phase)
    return wave

#########################################################
#    Complex multipler function:                        #
#    Input: I_1, Q_1, I_2, Q_2                          #
#    Output: Mixed wave in quadrature (I_mix, Q_mix)    #
#########################################################

def c_mult(I_1, Q_1, I_2, Q_2): 
    I_mix = I_1 * I_2 - Q_1 * Q_2 
    Q_mix = I_1 * Q_2 + Q_1 * I_2 
    return I_mix, Q_mix

def real_mix(wave_1, wave_2):
    mix_wave = wave_1 * wave_2
    return mix_wave

#########################################################
#    Other Functions:                                   #
#    FFT for I,Q signal                                 #
#    Noise Inclusion                                    # 
#    Chopping                                           #
#    Allan Variance                                    # 
#########################################################

def fft(signal):
    spectrum = np.fft.fft(signal, n = FFT_length)
    intensity = np.real(spectrum)**2 + np.imag(spectrum)**2
    return intensity

def fft_IQ(signal_i, signal_q):
    spectrum = np.fft.fft(signal_i + 1j*(signal_q), n = FFT_length)
    return spectrum

def noisify(signal_i, signal_q, POWER):
    w_noise = np.random.normal(0, POWER, signal_i.shape)
    noisy_sig_i = signal_i * w_noise
    w_noise = np.random.normal(0, POWER, signal.shape)
    noisy_sig_q = signal_q * w_noise
    return noisy_sig_i, noisy_sig_q

def GET_TO_DA_CHOPPAH(signal, timespace):
    sq_wave = 0.5 * (sig.square(2 * np.pi * square_freq * timespace) + 1)
    chopped_wave = real_mix(signal, sq_wave)
    return chopped_wave

def complex_choppin(sig_i, sig_q, timespace):
    sq_wave_i = 0.5 * (sig.square(2 * np.pi * square_freq * timespace) + 1) 
    sq_wave_q = 0.5 * (sig.square(2 * np.pi * square_freq * timespace + (np.pi/4)) + 1)
    chop_sig_i, chop_sig_q = cmult(sig_i, sig_q, sq_wave_i, sq_wave_q)
    return chop_sig_i, chop_sig_q
    
def save_data(file_name, data):
    file = open(file_name, 'w')
    writer = csv.writer(file)
    writer.writerow(data)
    
def allan_var(timestream, res=30):
    rate = 1/(accum_time)
    tau = np.logspace(0, run_time/5, res)
    (tau2, adevs, adev_err, n) = AT.oadev(timestream, rate, data_type="freq", taus=tau)
    avars = np.square(adevs)
    white_line = (avars[0]*(tau2**-1))
    return avars, white_line, adev_err, tau2    

In [6]:
# To check and see how modulation and demodulation work for quadrature chopped waves, lets make an example
# We will create a square wave signal similar to that which we would see from the PFB output stream
# Then we will do a complex multiply by a wave with I and Q at the same frequency as the chopper
# Examine the results in the time domain and frequency domain
samptime = np.linspace(0,10,1024)

fft_freq = np.fft.fftfreq(1024, d=10/1024)


sig_i, sig_q = cool_wave(2, 4, samptime)
#signal = real_wave(1, 4, samptime, 0)
sq_wave_i = 5 * sig.square(2 * np.pi * 1 * samptime) + 1 
#sq_wave_q = sig.square(2 * np.pi * 2 * samptime + (np.pi/4)) + 1
chop_i, chop_q = real_mix(sig_i, sq_wave_i), real_mix(sig_q, sq_wave_i)


cordic_i, cordic_q = cool_wave(2, 1, samptime)
mix_i, mix_q = c_mult(chop_i, chop_q, cordic_i, cordic_q)

downmix = fft(mix_i + 1j*(mix_q))

In [ ]:
fig, axs = plt.subplots(5, sharex=True)
#.plot(x, y)

axs[0].plot(fft_freq, fft(sig_i))

axs[1].plot(fft_freq, fft(sq_wave_i))

axs[2].plot(fft_freq, fft(cordic_i))

axs[3].plot(fft_freq, fft(chop_i))

axs[4].plot(fft_freq, downmix)
plt.xlim(0.1,10)

In [ ]:
# Lets do the same thing with noise now to make it a little more realistic
# The claim is that the lock in amplification will lower the minimum detectable power level
# Lets test this by making our incoming signal have a slider widget to determine power
# Then we will compare how much power is seen with and without lock-in

In [ ]:
# I believe this is the only way to prove to Mona that lock-in is worthless
# Doing a minimum detectable power test on both hoh_spec AND lock-in v_3
# TO do this I have to first actually create lock-in V_3 and prove that it works
# This can be done by taking the Allan variance (I guess) and comparing it to software sim
# Once lock-in V_3 is proven to be what mona asked for, then compare minimum detectable power
# My gut tells me that the hoh_spec firmware will have just as low of a MDP as lock-in, if not lower 

In [18]:
#***********************************#
#                                   #
#           Make FFT Frames         #        
#                                   #
#***********************************#

    ##### for a 1024 pt FFT, one FFT frame will take 1024/adc_clk (seconds) ##### 
.


    # frame time = Time for 1 FFT to populate
frame_time = FFT_length / (adc_clk)

    # frame_freq = how many FFT frames are created in one second
frame_freq = adc_clk/FFT_length

    #The number of fft frames to be created equals the run time of the test over the amount of time it takes to FFT a frame                        #

    # accum_frames = number of FFT frames created over one accumulation
accum_frames = int(accum_time/frame_time)

    # Now create array which contains frequency span of FFT in order to have correctly scaled FFT plots
#timestep = 1/adc_clk
    
    # note this value comes from the linspace used for the time array. length/number of samples
    # this array will be used to properly map the x-axis in Fourier space plots
#fft_freq = np.fft.fftfreq(FFT_length, d=timestep)


In [19]:
## Simulating the lock in (using modulated white noise data)

def lock_in(chan, run_time, phase=0):
        
        # Define how many seconds of data should be simulated and how many accumulations are required
    
    num_accum = int(run_time * (1/accum_time)) # number of total accumulations in simulation time
    
        # create file name for saving data in csv if desired
        
    file_name = '%d_sec_lockin_sim_chan_%d.csv'%(run_time, chan)    
    
        #Make an empty array to be filled with accumulations
        
    accums = np.zeros(num_accum)
    
    mix_mags = []

    #***********************************#
    #                                   #
    #           Creating Time           #        
    #                                   #
    #***********************************#

    for i in range(num_accum):
        print('we are on accumulation number %d out of %d'%(i, num_accum))
        
            ##### Create an array with the times of the FFT frames #####
            
        frame_times = np.linspace(i * frame_time, i * frame_time + (accum_frames-1) * frame_time, (accum_frames)  )
        
            ##### Create an array of times that will be used to create the "pieces" of the wave #####
            ##### Populate time array with lengths to be used later #####
            ##### This is an absolutely crazy vectorization of a previous loop I had, but it runs 100 times faster. Sorry #####
            
        timespace = np.linspace(np.linspace(frame_times[0], frame_times[1], FFT_length), 
                                np.linspace(frame_times[accum_frames-2], frame_times[accum_frames-1], FFT_length),
                                num = accum_frames-2)
        
            
    #***********************************#
    #                                   #
    #            Signals                #
    #   (Creation and Timestreaming)    #
    #                                   #
    #***********************************#
               
            # create empty array to hold white noise signal
        
        #signal = np.zeros(np.shape(timespace))
            
            # create white noise equivalent to what is used in tests
            
        #for j in range(accum_frames-1): signal[j] = allan.noise.white(FFT_length, 0.1, adc_clk) #create array of white noise          
            
        signal =np.random.normal(0, 0.1, size=np.shape(timespace)) 
       
            # Chop up white noise at chop frequency (comment next line to get unchopped results)

        choppa_signal = GET_TO_DA_CHOPPAH(signal, timespace)

            # Now put the choppeed noise signal through PFB
        
        spectra = fft(signal)

            # To get timestreams, convert from rows of freq and columns of time to rows of time streams and columns of freq
            # I'm pretty proud of how cheeky this trick is tbh, all you need is a transpose, no array slicing and dicing
            
        t_streams = np.transpose(spectra)
        t_streams_i = np.real(t_streams)
        t_streams_q = np.imag(t_streams)
        
        #########################################
        #    Mixing Channel Timestreams Down    #
        #########################################   
        
            # Create time array to control internally generated wave 

        timespace2 = np.linspace(0, run_time, accum_frames-1)

            # Create generated signal inside FPGA at square wave frequency 
    
        chop_i, chop_q = cool_wave(1, square_freq, timespace2)
        chop_i, chop_q = np.array(list(chop_i).pop()), np.array(list(chop_q).pop())
        
            # Mix together timestreams and chops
        
        downmix = c_mult(t_streams_i[chan], t_streams_q[chan], chop_i, chop_q)
        
        
        downmix_mag = np.real(downmix)**2 + np.imag(downmix)**2
        
            # Now integrate the timestream as is done in firmware
        
        accum = np.sum(downmix_mag)
            
            # Plop the accumulation data into our accumulation array for further faffing about
        
        accums[i] = accum

    return file_name, accums

In [ ]:
lock_in(100,1)

In [ ]:
#***********************************#
#                                   #
#           Widget Fun              #        
#                                   #
#***********************************#
sampspace = np.linspace(0,10,1000)
def real_wave1(amp, freq, time, phase):
    omega = 2 * np.pi * freq    
    wave = amp*np.cos(omega * time + phase) 
    #plt.plot(time, wave)
    return wave
def real_wave2(amp, freq, time, phase):
    omega = 2 * np.pi * freq    
    wave = amp*np.cos(omega * time + phase) 
    #plt.plot(time, wave)
    return wave

phase_widg = wdg.FloatSlider(min = -np.pi, step =.01, max = np.pi, value = 0)

widget = wdg.interactive(real_wave1, amp = wdg.fixed(1), freq = wdg.fixed(2), time = wdg.fixed(sampspace), 
                         phase=wdg.FloatSlider(min = -np.pi, step =.01, max = np.pi, value = 0))
display(widget)


In [ ]:
run_time = 2
num_accum = int(run_time * (1/accum_time)) # number of total accumulations in simulation time
    
        # create file name for saving data in csv if desired
        
#file_name = '%d_sec_lockin_sim_chan_%d.csv'%(run_time, chan)    
    
        #Make an empty array to be filled with accumulations
        
accums = np.zeros(num_accum)

for i in range(num_accum):
        #print('we are on accumulation number %d out of %d'%(i, num_accum))
        
            ##### Create an array with the times of the FFT frames #####
            
        frame_times = np.linspace(i * frame_time, i * frame_time + (accum_frames-1) * frame_time, (accum_frames))
        
            ##### Create an array of times that will be used to create the "pieces" of the wave #####

        timespace = np.zeros((accum_frames-1, FFT_length))
        

            ##### Populate time array with lengths to be used later #####

        for j in range(accum_frames-1):

            timespace[j] = np.linspace(frame_times[j], frame_times[j+1], FFT_length)
print(len(timespace[0]))